In [27]:
import warnings
import itertools
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from evaluate import evaluate

First let's read in the data, both the training and test sets. The test train split was done beforehand via the test-train-split script.

In [28]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

train_data.columns

Index(['Song', 'Song.ID', 'Artist', 'Album', 'Explicit', 'Playtime',
       'Loudness', 'Time.Signature', 'Popularity', 'Energy', 'Positiveness',
       'Speechiness', 'Liveliness', 'Acousticness', 'Instrumentalness',
       'Danceability', 'Tempo', 'Key', 'Mode', 'in_Blend', 'in_Bliss',
       'in_Rock', 'in_Rap', 'in_Oldies', 'in_Orchestral', 'in_Cinema',
       'TimeSignature_3', 'TimeSignature_5', 'TimeSignature_1', 'Age'],
      dtype='object')

In [64]:
ALL_FEATURES = ['Explicit', 'Playtime', 'Loudness', 'Popularity',
                'Energy', 'Positiveness', 'Speechiness', 'Liveliness', 'Acousticness',
                'Instrumentalness', 'Danceability', 'Tempo', 'Key', 'Mode','Age',
                'Time.Signature', 'TimeSignature_1', 'TimeSignature_3', 'TimeSignature_5']
ALL_FEATURES_WITHOUT_TIME = [feat for feat in ALL_FEATURES if 'Time' not in feat]
ALL_FEATURES_WITH_TIME_AS_CATEGORY = [feat for feat in ALL_FEATURES if feat != 'Time.Signature']
ALL_FEATURES_WITH_TIME_AS_NUMERIC = ALL_FEATURES_WITHOUT_TIME + ['Time.Signature']

In [54]:
def get_folds_for_playlist(features_to_include, K = 5, playlist='in_Rap', quiet=False):
    FOLDS = []
    fold_size = len(train_data) // K
    for i in range(K):
        val = train_data.iloc[i*fold_size: (i+1)*fold_size]
        train = pd.concat([train_data.iloc[:i*fold_size]
                           , train_data[(i+1)*fold_size:]], 
                          axis=0)
        X_train, y_train = train[features_to_include], train[playlist]
        y_train = y_train.to_numpy()
        X_val, y_val = val[features_to_include], val[playlist]
        y_val = y_val.to_numpy()
        FOLDS.append({
            'X_train': X_train,
            'y_train': y_train,
            'X_val': X_val,
            'y_val': y_val
        })
    if not quiet:
        print("Num folds:", K)
        print("Fold Xy Train Shape:", FOLDS[0]['X_train'].shape, FOLDS[0]['y_train'].shape)
        print("Fold Xy Val Shape:", FOLDS[0]['X_val'].shape, FOLDS[0]['y_val'].shape)
    return FOLDS

In [55]:
def get_feature_combos(feature_list=ALL_FEATURES, necessary_features=[], minimum_number_features = 4):
    candidate_combinations = []

    # Get all possible feature combinations
    for k in range(minimum_number_features, len(feature_list) + 1):
        combinations_of_size_k = list(itertools.combinations(feature_list, k))
        candidate_combinations.extend(combinations_of_size_k)
    candidate_combinations = [list(combo) for combo in candidate_combinations]

    final_combinations = []
    # Filter combinations for only good combos
    for comb in candidate_combinations:
        # skip this combination if it doesn't have
        # all the necessary features.
        has_all_necessary = all([feat in comb for feat in necessary_features])
        if not has_all_necessary:
            continue # continue to next combination

        # time signiature should be completely used or not at all
        num_time_sig_feats = len([feat for feat in comb if 'Time' in feat])
        if num_time_sig_feats == 0:
            final_combinations.append(comb)
        # If there are time signatures in there,
        # Keep this combination only if it's the single numerical feature,
        # or the 3 categories.
        else:
            if num_time_sig_feats == 1 and 'Time.Signature' in comb:
                final_combinations.append(comb)
            elif num_time_sig_feats == 3 and 'Time.Signature' not in comb:
                final_combinations.append(comb)
    
    return final_combinations

Now let's build a cross-validation environment

In [56]:
def cv_workflow(hyperparams_to_try, feature_sets_to_try, playlist, K=5, quiet=False):
    # Loops through hyperparameter options
    # and returns the hyperparameters that achieved
    # the best k-fold avg f1 score, along with the score.
    def eval_model_on_folds(model, quiet=False):
        scaler = StandardScaler()
        
        f1_scores = []
        for i, fold in enumerate(FOLDS):
            X_train, y_train = fold['X_train'], fold['y_train']
            X_val, y_val = fold['X_val'], fold['y_val']

            # Standardize
            X_train = scaler.fit_transform(X_train)
            # Apply same transformation to validation set
            X_val = scaler.transform(X_val)
                
            # Train and evaluate
            model.fit(X_train, y_train)
            preds = model.predict(X_val)
            
            f1_scores.append(evaluate(preds, y_val, quiet=quiet))
        avg = np.mean(f1_scores)
        if not quiet:
            print(f"Avg f1 score over {len(FOLDS)} folds: {avg}")
        return avg
    
    if not quiet:
        print(f"===Performing {K}-fold cross validation for playlist '{playlist}'===")
    
    best_score = -1
    best_params = {}
    best_feats = []
    total_iterations = len(hyperparams_to_try) * len(feature_sets_to_try)
    it = 1
    for feature_list in feature_sets_to_try:
        FOLDS = get_folds_for_playlist(feature_list, K=K, playlist=playlist, quiet=quiet)
        for params_dict in hyperparams_to_try:
            if not quiet:
                print(f"{it}/{total_iterations}")
            model = LogisticRegression(**params_dict)
            k_fold_f1_score = eval_model_on_folds(model, quiet=quiet)
            if k_fold_f1_score > best_score:
                if not quiet:
                    print("Updating best model:", params_dict, feature_list)
                best_score = k_fold_f1_score
                best_params = params_dict
                best_feats = feature_list
            it += 1
    if not quiet:
        print(f"Best {K}-fold f1 score was", best_score)
        print("Best model were", best_params, feature_list)
    return best_params, best_score, best_feats

In [57]:
base_params = {'penalty': 'l1', 'solver': 'liblinear'}
hyperparams_to_try = [
    {**base_params, 'C':0.001}, 
    {**base_params, 'C':0.01},
    {**base_params, 'C':0.1},
    {**base_params, 'C':1},
    {**base_params, 'C':10},
    {**base_params, 'C':100},
    {**base_params, 'C':10e6}, # Almost no regularization
    {'penalty': None, 'solver': 'newton-cg'}, # No regularization    
]

## Rap Playlist ##

Let's start with the rap playlist, since it should be fairly predictable.
Note that some features like explicitness are nearly
perfect predictors of being in a rap playlist, so we need to make sure to 
regularize since perfect separation is a problem for logistic regression.
First we get our response variables for test and train sets.

In [34]:
y_train_rap = train_data['in_Rap'].to_numpy()
y_val_rap = test_data['in_Rap'].to_numpy()

### Manual Model Selection

#### Manual Model 1: Using Only Very Highly Predictive Features From Exploratory Data Analysis

In [48]:
chosen_features_rap_manual = ['Explicit', 'Instrumentalness', 'Speechiness']
_, f1_rap, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Rap', feature_sets_to_try=[chosen_features_rap_manual], K=5, quiet=True)
X_train_rap_manual = train_data[chosen_features_rap_manual]
X_val_rap_manual = test_data[chosen_features_rap_manual]

mod_rap_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_rap_manual.fit(X_train_rap_manual, y_train_rap)
coefs_rap = pd.DataFrame({"Feature": chosen_features_rap_manual, "Coef": mod_rap_manual.coef_[0], "Abs_Coef": abs(mod_rap_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_rap}")
coefs_rap[["Feature", "Coef"]]

F1-Score: 0.9233174907005915


,Feature,Coef
0,Explicit,5.101962
2,Speechiness,0.099286
1,Instrumentalness,-0.043956


So the cross validation F1 score is 92.3%. With so few features this performance is quite good but there is still room to improve. We will try adding some other features to see if we can improve the model.

#### Manual Model 2: Adding Highly Predictive Features From Exploratory Data Analysis

In [49]:
chosen_features_rap_manual = ['Explicit', 'Instrumentalness', 'Speechiness', 'Loudness', 'Danceability']
_, f1_rap, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Rap', feature_sets_to_try=[chosen_features_rap_manual], K=5, quiet=True)
X_train_rap_manual = train_data[chosen_features_rap_manual]
X_val_rap_manual = test_data[chosen_features_rap_manual]

mod_rap_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_rap_manual.fit(X_train_rap_manual, y_train_rap)
coefs_rap = pd.DataFrame({"Feature": chosen_features_rap_manual, "Coef": mod_rap_manual.coef_[0], "Abs_Coef": abs(mod_rap_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_rap}")
coefs_rap[["Feature", "Coef"]]

F1-Score: 0.9264982140448424


,Feature,Coef
0,Explicit,5.143316
2,Speechiness,0.096463
4,Danceability,0.090339
3,Loudness,-0.077170
1,Instrumentalness,-0.031446


The cross validation performance has improved marginally. Explicit is easily the most important predictor but Danceability seems to be a decently significant predictor, passing Instrumentalness and almost reaching Speechiness. We will see if we can do even better by adding more features.

#### Manual Model 3: Using All Features From Exploratory Data Analysis (Excluding Time Signature)

In [50]:
chosen_features_rap_manual = ['Explicit', 'Instrumentalness', 'Speechiness', 'Loudness', 'Danceability', 'Popularity', 'Energy', 'Positiveness', 'Acousticness', 'Age']
_, f1_rap, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Rap', feature_sets_to_try=[chosen_features_rap_manual], K=5, quiet=True)
X_train_rap_manual = train_data[chosen_features_rap_manual]
X_val_rap_manual = test_data[chosen_features_rap_manual]

mod_rap_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_rap_manual.fit(X_train_rap_manual, y_train_rap)
coefs_rap = pd.DataFrame({"Feature": chosen_features_rap_manual, "Coef": mod_rap_manual.coef_[0], "Abs_Coef": abs(mod_rap_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_rap}")
coefs_rap[["Feature", "Coef"]]

F1-Score: 0.9227664714066902


,Feature,Coef
0,Explicit,4.768452
3,Loudness,0.150577
4,Danceability,0.119461
2,Speechiness,0.111418
7,Positiveness,-0.036989
1,Instrumentalness,-0.031304
5,Popularity,-0.023083
6,Energy,-0.014348
8,Acousticness,0.012422
9,Age,-0.005450


The cross validation performance has now dropped and we see that the coefficients for some of the features are near 0 so we will now attempt to drop those and include the rest.

#### Manual Model 4: Dropping Age, Acousticness, Energy, and Popularity

In [51]:
chosen_features_rap_manual = ['Explicit', 'Instrumentalness', 'Speechiness', 'Loudness', 'Danceability', 'Positiveness']
_, f1_rap, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Rap', feature_sets_to_try=[chosen_features_rap_manual], K=5, quiet=True)
X_train_rap_manual = train_data[chosen_features_rap_manual]
X_val_rap_manual = test_data[chosen_features_rap_manual]

mod_rap_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_rap_manual.fit(X_train_rap_manual, y_train_rap)
coefs_rap = pd.DataFrame({"Feature": chosen_features_rap_manual, "Coef": mod_rap_manual.coef_[0], "Abs_Coef": abs(mod_rap_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_rap}")
coefs_rap[["Feature", "Coef"]]

F1-Score: 0.9277559133373087


,Feature,Coef
0,Explicit,4.697282
4,Danceability,0.110292
2,Speechiness,0.102233
5,Positiveness,-0.049203
1,Instrumentalness,-0.032026
3,Loudness,0.030939


Our cross validation performance has now improved. This however goes against what we saw in our exploratory data analysis since Age is likely a good predictor (rap songs are not very old in general). What we are likely seeing is that these features are explanatory but the model is overfitting to the training data. In order to improve the model we will need to regularize it.

The idea behind adding time signature and tempo is that many rap songs are very fast so these could potentially be good predictors. We see that the performance from cross validation has improved even more. Before calling this our best manually selected model we will try removing the tempo feature to see if it is actually a good predictor.

### Automatically Determining Model Based on CV Performance

First testing all combinations of features with no regularization.

In [61]:
necessary_rap = ['Explicit', 'Danceability', 'Speechiness', 'Age']
feature_combos_rap = get_feature_combos(necessary_features=necessary_rap)
best_params_rap, best_score_rap, selected_feats = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], feature_combos_rap, playlist='in_Rap', K=5, quiet=True)
print("Best features:", selected_feats)
print("Best score:", best_score_rap)

Best features: ['Explicit', 'Playtime', 'Energy', 'Speechiness', 'Danceability', 'Mode', 'Age', 'TimeSignature_1', 'TimeSignature_3', 'TimeSignature_5']
Best score: 0.9482196265506799


Then testing all features with regularization.

In [66]:
best_params_rap, best_score_rap, best_lasso_features = cv_workflow(hyperparams_to_try, [ALL_FEATURES_WITH_TIME_AS_CATEGORY, ALL_FEATURES_WITH_TIME_AS_NUMERIC], playlist='in_Rap', K=5, quiet=True)
print("Best features:", best_lasso_features)
print("Best score:", best_score_rap)

Best features: ['Explicit', 'Playtime', 'Loudness', 'Popularity', 'Energy', 'Positiveness', 'Speechiness', 'Liveliness', 'Acousticness', 'Instrumentalness', 'Danceability', 'Tempo', 'Key', 'Mode', 'Age', 'Time.Signature']
Best score: 0.9274628061996483


Okay, we found the best lasso regularization strength. Let's see which features it deemed important.
We retrain on all the training data

In [69]:
X_train, y_train = train_data[best_lasso_features], train_data['in_Rap']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

rap_model = LogisticRegression(**best_params_rap)
rap_model.fit(X_train_scaled, y_train)
coefs = pd.DataFrame({"Feature": best_lasso_features, "Coef": rap_model.coef_[0], "Abs_Coef": abs(rap_model.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)
lasso_most_important_feats = coefs[["Feature"]].iloc[:len(selected_feats)].to_numpy()
agree = set(lasso_most_important_feats[:,0]) & set(selected_feats)
agree

{'Age', 'Danceability', 'Explicit', 'Speechiness'}

Now let's try running Lasso on top of the best selected features from best subset selection

In [72]:
best_params_rap, best_f1, _ = cv_workflow(hyperparams_to_try, [selected_feats], playlist='in_Rap', K=5, quiet=True)
print("Best features:", selected_feats)
print("Best params:", best_params_rap)
print("Best score:", best_f1)

Best features: ['Explicit', 'Playtime', 'Energy', 'Speechiness', 'Danceability', 'Mode', 'Age', 'TimeSignature_1', 'TimeSignature_3', 'TimeSignature_5']
Best params: {'penalty': 'l1', 'solver': 'liblinear', 'C': 10000000.0}
Best score: 0.9526640709951243


It looks like best subset with practically no regularization did the best, so let's evaluate it's test set performance

In [74]:
X_train, y_train = train_data[selected_feats], train_data['in_Rap']
X_val, y_val = test_data[selected_feats], test_data['in_Rap']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

rap_model = LogisticRegression(**{'penalty': None})
rap_model.fit(X_train_scaled, y_train)

X_test, y_test = test_data[selected_feats], test_data['in_Rap'].to_numpy()
# Apply same transformation to validation set
X_test_scaled = scaler.transform(X_val)
preds = rap_model.predict(X_test_scaled)

evaluate(preds, y_test)

Accuracy: 96.88% (124/128)
Num Predicted in Playlist/Num In playlist: 100.0% (30/30)
Num in Playlist/Num Predicted In playlist: 88.24% (30/34)
F1 Score: 93.75%


np.float64(0.9375)

## Bliss Playlist ##

Now we will continue with the bliss playlist, since it will be much less predictable.

In [76]:
y_train_bliss = train_data['in_Bliss'].to_numpy()
y_val_bliss = test_data['in_Bliss'].to_numpy()

### Manual Model Selection

For this playlist we know from the exploratory data analysis that it will be much less predictable so we will just start by using all the features from the exploratory data analysis.

#### Manual Model 1: Using All Predictive Features From Exploratory Data Analysis

In [77]:
chosen_features_bliss_manual = ['Instrumentalness', 'Explicit', 'Energy', 'Positiveness', 'Speechiness', 'Acousticness', 'Danceability']
_, f1_bliss, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Bliss', feature_sets_to_try=[chosen_features_bliss_manual], K=5, quiet=True)
X_train_bliss_manual = train_data[chosen_features_bliss_manual]
X_val_bliss_manual = test_data[chosen_features_bliss_manual]

mod_bliss_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_bliss_manual.fit(X_train_bliss_manual, y_train_bliss)
coefs_bliss = pd.DataFrame({"Feature": chosen_features_bliss_manual, "Coef": mod_bliss_manual.coef_[0], "Abs_Coef": abs(mod_bliss_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_bliss}")
coefs_bliss[["Feature", "Coef"]]

F1-Score: 0.3740234428240426


,Feature,Coef
1,Explicit,-2.778181
4,Speechiness,-0.170482
6,Danceability,0.048839
0,Instrumentalness,-0.045538
3,Positiveness,-0.027743
2,Energy,-0.007367
5,Acousticness,0.003361


The cross validation performance here is rather low. Before we remove any features we will just use all features and see which ones should be removed based on the coefficients.

#### Manual Model 2: Using All Features (Time Signature as Categorical)

In [81]:
chosen_features_bliss_manual = ALL_FEATURES_WITH_TIME_AS_CATEGORY
_, f1_bliss, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Bliss', feature_sets_to_try=[chosen_features_bliss_manual], K=5, quiet=True)
X_train_bliss_manual = train_data[chosen_features_bliss_manual]
X_val_bliss_manual = test_data[chosen_features_bliss_manual]

mod_bliss_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_bliss_manual.fit(X_train_bliss_manual, y_train_bliss)
coefs_bliss = pd.DataFrame({"Feature": chosen_features_bliss_manual, "Coef": mod_bliss_manual.coef_[0], "Abs_Coef": abs(mod_bliss_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_bliss}")
coefs_bliss[["Feature", "Coef"]]

F1-Score: 0.40452525252525257


c:\Users\mmsmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
c:\Users\mmsmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\optimize.py:100: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
c:\Users\mmsmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\optimize.py:319: ConvergenceWarning: newton-cg failed to converge at loss = 0.30916516744173783. Increase the number of iterations.
  warnings.warn(


,Feature,Coef
0,Explicit,-2.838973
17,TimeSignature_5,1.655751
16,TimeSignature_3,1.099630
15,TimeSignature_1,-0.845143
6,Speechiness,-0.163955
13,Mode,0.072853
9,Instrumentalness,-0.051147
10,Danceability,0.048040
2,Loudness,-0.046787
5,Positiveness,-0.024067


The performance has improved. Before removing any features we will try Time Signature as a numeric feature to see if that improves performance.

#### Manual Model 3: Using All Features (Time Signature as Numeric)

In [82]:
chosen_features_bliss_manual = ALL_FEATURES_WITH_TIME_AS_NUMERIC
_, f1_bliss, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Bliss', feature_sets_to_try=[chosen_features_bliss_manual], K=5, quiet=True)
X_train_bliss_manual = train_data[chosen_features_bliss_manual]
X_val_bliss_manual = test_data[chosen_features_bliss_manual]

mod_bliss_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_bliss_manual.fit(X_train_bliss_manual, y_train_bliss)
coefs_bliss = pd.DataFrame({"Feature": chosen_features_bliss_manual, "Coef": mod_bliss_manual.coef_[0], "Abs_Coef": abs(mod_bliss_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_bliss}")
coefs_bliss[["Feature", "Coef"]]

F1-Score: 0.3887038123167156


c:\Users\mmsmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
c:\Users\mmsmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\optimize.py:100: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
c:\Users\mmsmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\optimize.py:319: ConvergenceWarning: newton-cg failed to converge at loss = 0.3145902459979145. Increase the number of iterations.
  warnings.warn(


,Feature,Coef
0,Explicit,-2.933271
15,Time.Signature,-0.247965
6,Speechiness,-0.155204
13,Mode,0.102120
9,Instrumentalness,-0.050893
2,Loudness,-0.046230
10,Danceability,0.045099
5,Positiveness,-0.025018
7,Liveliness,-0.019763
3,Popularity,0.007503


It seems like Time Signature as a categorical feature is a better predictor. We will now try removing the features with low coefficient values and treating Time Signature as a categorical feature to see if that improves performance.

#### Manual Model 4: Dropping Playtime, Tempo, Age, Key, Energy, Acousticness, and Popularity (Time Signature as Categorical) 

We still have the same performance without regularization. This is a good sign that we are not overfitting and that the model is generalizing well. We will now try to add some more features to see if we can improve the performance even more.

In [84]:
chosen_features_bliss_manual = [feature for feature in ALL_FEATURES_WITH_TIME_AS_CATEGORY if feature not in ['Playtime', 'Tempo', 'Age', 'Key', 'Energy', 'Acousticness', 'Popularity']]
_, f1_bliss, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Bliss', feature_sets_to_try=[chosen_features_bliss_manual], K=5, quiet=True)
X_train_bliss_manual = train_data[chosen_features_bliss_manual]
X_val_bliss_manual = test_data[chosen_features_bliss_manual]

mod_bliss_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_bliss_manual.fit(X_train_bliss_manual, y_train_bliss)
coefs_bliss = pd.DataFrame({"Feature": chosen_features_bliss_manual, "Coef": mod_bliss_manual.coef_[0], "Abs_Coef": abs(mod_bliss_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_bliss}")
coefs_bliss[["Feature", "Coef"]]

F1-Score: 0.4560224089635854


,Feature,Coef
8,TimeSignature_1,-3.223066
0,Explicit,-2.608037
10,TimeSignature_5,1.744086
9,TimeSignature_3,1.150681
3,Speechiness,-0.172395
7,Mode,0.052178
6,Danceability,0.051035
5,Instrumentalness,-0.049715
1,Loudness,-0.035848
2,Positiveness,-0.028100


The cross validation score has improved dramatically. Lets see if we can remove any more features to improve performance. Note that likely what we are seeing here is again that the model is overfitting to the training data amd we will need to regularize to improve performance.

#### Manual Model 5: Dropping Liveliness, Positiveness, Loudness (Time Signature as Categorical)

In [87]:
chosen_features_bliss_manual = [feature for feature in ALL_FEATURES_WITH_TIME_AS_CATEGORY if feature not in ['Playtime', 'Tempo', 'Age', 'Key', 'Energy', 'Acousticness', 'Popularity', 'Liveliness']]
_, f1_bliss, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Bliss', feature_sets_to_try=[chosen_features_bliss_manual], K=5, quiet=True)
X_train_bliss_manual = train_data[chosen_features_bliss_manual]
X_val_bliss_manual = test_data[chosen_features_bliss_manual]

mod_bliss_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_bliss_manual.fit(X_train_bliss_manual, y_train_bliss)
coefs_bliss = pd.DataFrame({"Feature": chosen_features_bliss_manual, "Coef": mod_bliss_manual.coef_[0], "Abs_Coef": abs(mod_bliss_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_bliss}")
coefs_bliss[["Feature", "Coef"]]

F1-Score: 0.4575860566448801


,Feature,Coef
7,TimeSignature_1,-2.664462
0,Explicit,-2.609872
9,TimeSignature_5,1.868641
8,TimeSignature_3,1.114133
3,Speechiness,-0.187997
5,Danceability,0.053654
4,Instrumentalness,-0.049553
1,Loudness,-0.041045
2,Positiveness,-0.028655
6,Mode,0.026459


The performance has slightly increased, this is likely the best we can get manually. So we will stick with model 5 as the best from our manual selection process and move on to automatic model selection.

### Automatically Determining Model Based on CV Performance

First testing all combinations of features with no regularization.

In [88]:
necessary_bliss = ['Instrumentalness', 'Danceability']
feature_combos_bliss = get_feature_combos(necessary_features=necessary_bliss)
best_params_bliss, best_score_bliss, selected_feats = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], feature_combos_bliss, playlist='in_Bliss', K=5, quiet=True)
print("Best features:", selected_feats)
print("Best score:", best_score_bliss)

c:\Chrome-Downloads\Code\stat-230a-project\evaluate.py:47: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * prec_pct * rec_pct)/(prec_pct + rec_pct)
c:\Chrome-Downloads\Code\stat-230a-project\evaluate.py:47: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * prec_pct * rec_pct)/(prec_pct + rec_pct)
c:\Chrome-Downloads\Code\stat-230a-project\evaluate.py:47: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * prec_pct * rec_pct)/(prec_pct + rec_pct)
c:\Chrome-Downloads\Code\stat-230a-project\evaluate.py:47: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * prec_pct * rec_pct)/(prec_pct + rec_pct)
c:\Chrome-Downloads\Code\stat-230a-project\evaluate.py:47: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * prec_pct * rec_pct)/(prec_pct + rec_pct)
c:\Chrome-Downloads\Code\stat-230a-project\evaluate.py:47: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * prec_pct * rec_

Best features: ['Explicit', 'Loudness', 'Popularity', 'Energy', 'Positiveness', 'Speechiness', 'Liveliness', 'Instrumentalness', 'Danceability', 'Key', 'TimeSignature_1', 'TimeSignature_3', 'TimeSignature_5']
Best score: 0.4853294853294853


All of the warnings are from the models predicting 0% correct so we get a divide by 0 warning. Then testing all features with regularization.

In [89]:
best_params_bliss, best_score_bliss, best_lasso_features = cv_workflow(hyperparams_to_try, [ALL_FEATURES_WITH_TIME_AS_CATEGORY, ALL_FEATURES_WITH_TIME_AS_NUMERIC], playlist='in_Bliss', K=5, quiet=True)
print("Best features:", best_lasso_features)
print("Best score:", best_score_bliss)

Best features: ['Explicit', 'Playtime', 'Loudness', 'Popularity', 'Energy', 'Positiveness', 'Speechiness', 'Liveliness', 'Acousticness', 'Instrumentalness', 'Danceability', 'Tempo', 'Key', 'Mode', 'Age', 'TimeSignature_1', 'TimeSignature_3', 'TimeSignature_5']
Best score: 0.413437908496732


Okay, we found the best lasso regularization strength. Let's see which features it deemed important.
We retrain on all the training data

In [90]:
X_train, y_train = train_data[best_lasso_features], train_data['in_Bliss']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

bliss_model = LogisticRegression(**best_params_bliss)
bliss_model.fit(X_train_scaled, y_train)
coefs = pd.DataFrame({"Feature": best_lasso_features, "Coef": bliss_model.coef_[0], "Abs_Coef": abs(bliss_model.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)
lasso_most_important_feats = coefs[["Feature"]].iloc[:len(selected_feats)].to_numpy()
agree = set(lasso_most_important_feats[:,0]) & set(selected_feats)
agree

{'Danceability',
 'Explicit',
 'Instrumentalness',
 'Liveliness',
 'Loudness',
 'Popularity',
 'Positiveness',
 'Speechiness',
 'TimeSignature_1',
 'TimeSignature_3',
 'TimeSignature_5'}

Now let's try running Lasso on top of the best selected features from best subset selection

In [91]:
best_params_bliss, best_f1, _ = cv_workflow(hyperparams_to_try, [selected_feats], playlist='in_Bliss', K=5, quiet=True)
print("Best features:", selected_feats)
print("Best params:", best_params_bliss)
print("Best score:", best_f1)

Best features: ['Explicit', 'Loudness', 'Popularity', 'Energy', 'Positiveness', 'Speechiness', 'Liveliness', 'Instrumentalness', 'Danceability', 'Key', 'TimeSignature_1', 'TimeSignature_3', 'TimeSignature_5']
Best params: {'penalty': 'l1', 'solver': 'liblinear', 'C': 100}
Best score: 0.4853294853294853


It looks like best subset with some regularization did the best, so let's evaluate it's test set performance

In [92]:
X_train, y_train = train_data[selected_feats], train_data['in_Bliss']
X_val, y_val = test_data[selected_feats], test_data['in_Bliss']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

bliss_model = LogisticRegression(**{'penalty': None})
bliss_model.fit(X_train_scaled, y_train)

X_test, y_test = test_data[selected_feats], test_data['in_Bliss'].to_numpy()
# Apply same transformation to validation set
X_test_scaled = scaler.transform(X_val)
preds = bliss_model.predict(X_test_scaled)

evaluate(preds, y_test)

Accuracy: 75.78% (97/128)
Num Predicted in Playlist/Num In playlist: 25.0% (8/32)
Num in Playlist/Num Predicted In playlist: 53.33% (8/15)
F1 Score: 34.04%


np.float64(0.3404255319148936)

## Blend Playlist ##

Now we will continue with the Blend playlist, since it is likely going to be a middle ground between the predictability of the other two playlists.

In [94]:
y_train_blend = train_data['in_Blend'].to_numpy()
y_val_blend = test_data['in_Blend'].to_numpy()

### Manual Model Selection

#### Manual Model 1: Using Only Highly Predictive Features From Exploratory Data Analysis

In [96]:
chosen_features_blend_manual = ['Instrumentalness', 'Positiveness', 'Energy']
_, f1_blend, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Blend', feature_sets_to_try=[chosen_features_blend_manual], K=5, quiet=True)
X_train_blend_manual = train_data[chosen_features_blend_manual]
X_val_blend_manual = test_data[chosen_features_blend_manual]

mod_blend_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_blend_manual.fit(X_train_blend_manual, y_train_blend)
coefs_blend = pd.DataFrame({"Feature": chosen_features_blend_manual, "Coef": mod_blend_manual.coef_[0], "Abs_Coef": abs(mod_blend_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_blend}")
coefs_blend[["Feature", "Coef"]]

F1-Score: 0.6119898441509332


,Feature,Coef
0,Instrumentalness,-0.039996
1,Positiveness,-0.012512
2,Energy,-0.005341


The cross validation here isn't horrible but we would like to improve it. We will remove Energy and add the rest of the features from our exploratory analysis to see if that improves performance.

#### Manual Model 2: Adding Predictive Features From Exploratory Data Analysis And Dropping Energy

In [97]:
chosen_features_blend_manual = ['Instrumentalness', 'Positiveness', 'Explicit', 'Loudness', 'Acousticness', 'Danceability']
_, f1_blend, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Blend', feature_sets_to_try=[chosen_features_blend_manual], K=5, quiet=True)
X_train_blend_manual = train_data[chosen_features_blend_manual]
X_val_blend_manual = test_data[chosen_features_blend_manual]

mod_blend_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_blend_manual.fit(X_train_blend_manual, y_train_blend)
coefs_blend = pd.DataFrame({"Feature": chosen_features_blend_manual, "Coef": mod_blend_manual.coef_[0], "Abs_Coef": abs(mod_blend_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_blend}")
coefs_blend[["Feature", "Coef"]]

F1-Score: 0.6430343611183694


,Feature,Coef
2,Explicit,-2.029849
0,Instrumentalness,-0.039846
5,Danceability,0.030633
1,Positiveness,-0.029339
3,Loudness,0.019737
4,Acousticness,-0.004734


The performance has improved but we still would like to do better. Clearly Acousticness is not a good predictor so we will remove it and try adding every other feature except for Energy, first treating Time Signature as a categorical feature.

#### Manual Model 3: Using All Features And Dropping Loudness (Time Signature as Categorical)

In [98]:
chosen_features_blend_manual = [feature for feature in ALL_FEATURES_WITH_TIME_AS_CATEGORY if feature not in ['Time.Signature', 'Acousticness', 'Energy']]
_, f1_blend, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Blend', feature_sets_to_try=[chosen_features_blend_manual], K=5, quiet=True)
X_train_blend_manual = train_data[chosen_features_blend_manual]
X_val_blend_manual = test_data[chosen_features_blend_manual]

mod_blend_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_blend_manual.fit(X_train_blend_manual, y_train_blend)
coefs_blend = pd.DataFrame({"Feature": chosen_features_blend_manual, "Coef": mod_blend_manual.coef_[0], "Abs_Coef": abs(mod_blend_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_blend}")
coefs_blend[["Feature", "Coef"]]

F1-Score: 0.609317364719655


c:\Users\mmsmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
c:\Users\mmsmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\optimize.py:100: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(


,Feature,Coef
0,Explicit,-2.453626
15,TimeSignature_5,0.270086
14,TimeSignature_3,-0.107462
13,TimeSignature_1,0.100561
2,Loudness,0.041692
7,Instrumentalness,-0.038687
8,Danceability,0.032610
4,Positiveness,-0.030405
11,Mode,0.016609
3,Popularity,0.015756


The cross validation performance has now dropped and we see that the coefficients for some many of the features are near 0 so we will remove all of those features and check the performance again. This again seems like a scenario where we need to regularize to improve performance.

#### Manual Model 4: Dropping Insignificant Features

We still have the same performance without regularization. This is a good sign that we are not overfitting and that the model is generalizing well. We will now try to add some more features to see if we can improve the performance even more.

In [99]:
chosen_features_blend_manual = [feature for feature in ALL_FEATURES_WITH_TIME_AS_CATEGORY if feature not in ['Time.Signature', 'Acousticness', 'Energy', 'Playtime', 'Age', 'Liveliness', 'Key', 'Tempo']]
_, f1_blend, _ = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], playlist='in_Blend', feature_sets_to_try=[chosen_features_blend_manual], K=5, quiet=True)
X_train_blend_manual = train_data[chosen_features_blend_manual]
X_val_blend_manual = test_data[chosen_features_blend_manual]

mod_blend_manual = LogisticRegression(**{'penalty': None, 'solver': 'newton-cg'})
mod_blend_manual.fit(X_train_blend_manual, y_train_blend)
coefs_blend = pd.DataFrame({"Feature": chosen_features_blend_manual, "Coef": mod_blend_manual.coef_[0], "Abs_Coef": abs(mod_blend_manual.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)

print(f"F1-Score: {f1_blend}")
coefs_blend[["Feature", "Coef"]]

F1-Score: 0.6120198433499707


,Feature,Coef
0,Explicit,-2.416498
9,TimeSignature_3,-0.192756
10,TimeSignature_5,0.092682
7,Mode,-0.060752
8,TimeSignature_1,0.052518
1,Loudness,0.045050
5,Instrumentalness,-0.038738
3,Positiveness,-0.030049
6,Danceability,0.028743
2,Popularity,0.015238


The performance still isn't as good as what we saw before. Again it seems like we need regularization so we will move on to automatic model selection.

### Automatically Determining Model Based on CV Performance

First testing all combinations of features with no regularization.

In [101]:
necessary_blend = ['Positiveness', 'Instrumentalness']
feature_combos_blend = get_feature_combos(necessary_features=necessary_blend)
best_params_blend, best_score_blend, selected_feats = cv_workflow([{'penalty': None, 'solver': 'newton-cg'}], feature_combos_blend, playlist='in_Blend', K=5, quiet=True)
print("Best features:", selected_feats)
print("Best score:", best_score_blend)

Best features: ['Explicit', 'Popularity', 'Energy', 'Positiveness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Tempo', 'Key']
Best score: 0.6754100646279539


All of the warnings are from the models predicting 0% correct so we get a divide by 0 warning. Then testing all features with regularization.

In [102]:
best_params_blend, best_score_blend, best_lasso_features = cv_workflow(hyperparams_to_try, [ALL_FEATURES_WITH_TIME_AS_CATEGORY, ALL_FEATURES_WITH_TIME_AS_NUMERIC], playlist='in_Blend', K=5, quiet=True)
print("Best features:", best_lasso_features)
print("Best score:", best_score_blend)

Best features: ['Explicit', 'Playtime', 'Loudness', 'Popularity', 'Energy', 'Positiveness', 'Speechiness', 'Liveliness', 'Acousticness', 'Instrumentalness', 'Danceability', 'Tempo', 'Key', 'Mode', 'Age', 'TimeSignature_1', 'TimeSignature_3', 'TimeSignature_5']
Best score: 0.6642053572854802


Okay, we found the best lasso regularization strength. Let's see which features it deemed important.
We retrain on all the training data

In [103]:
X_train, y_train = train_data[best_lasso_features], train_data['in_Blend']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

blend_model = LogisticRegression(**best_params_blend)
blend_model.fit(X_train_scaled, y_train)
coefs = pd.DataFrame({"Feature": best_lasso_features, "Coef": blend_model.coef_[0], "Abs_Coef": abs(blend_model.coef_[0])}).sort_values(by="Abs_Coef", ascending=False)
lasso_most_important_feats = coefs[["Feature"]].iloc[:len(selected_feats)].to_numpy()
agree = set(lasso_most_important_feats[:,0]) & set(selected_feats)
agree

{'Acousticness',
 'Energy',
 'Explicit',
 'Instrumentalness',
 'Key',
 'Popularity',
 'Positiveness'}

Now let's try running Lasso on top of the best selected features from best subset selection

In [104]:
best_params_blend, best_f1, _ = cv_workflow(hyperparams_to_try, [selected_feats], playlist='in_Blend', K=5, quiet=True)
print("Best features:", selected_feats)
print("Best params:", best_params_blend)
print("Best score:", best_f1)

Best features: ['Explicit', 'Popularity', 'Energy', 'Positiveness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Tempo', 'Key']
Best params: {'penalty': 'l1', 'solver': 'liblinear', 'C': 10}
Best score: 0.6754100646279539


It looks like best subset with some regularization did the best, so let's evaluate it's test set performance

In [105]:
X_train, y_train = train_data[selected_feats], train_data['in_Blend']
X_val, y_val = test_data[selected_feats], test_data['in_Blend']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

blend_model = LogisticRegression(**{'penalty': None})
blend_model.fit(X_train_scaled, y_train)

X_test, y_test = test_data[selected_feats], test_data['in_Blend'].to_numpy()
# Apply same transformation to validation set
X_test_scaled = scaler.transform(X_val)
preds = blend_model.predict(X_test_scaled)

evaluate(preds, y_test)

Accuracy: 72.66% (93/128)
Num Predicted in Playlist/Num In playlist: 59.7% (40/67)
Num in Playlist/Num Predicted In playlist: 83.33% (40/48)
F1 Score: 69.57%


np.float64(0.6956521739130433)